# 这部分的experiments trading day only (因为more reasonable)

In [1]:
import pandas as pd
import numpy as np
import os
import tqdm
from scipy import interpolate
import OptionTools
import re
import datetime
import time
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [2]:
import statsmodels.api as sm

def OLS_evaluate(df, col):
    df = df.dropna()
    MON = df['weekday'].apply(lambda x: 1 if x==0 else 0).to_numpy()
    TUE = df['weekday'].apply(lambda x: 1 if x==1 else 0).to_numpy()
    WED = df['weekday'].apply(lambda x: 1 if x==2 else 0).to_numpy()
    THU = df['weekday'].apply(lambda x: 1 if x==3 else 0).to_numpy()
    FRI = df['weekday'].apply(lambda x: 1 if x==4 else 0).to_numpy()
    y = df[col].values
    X = np.column_stack([MON, TUE, WED, THU, FRI])
    if add:
        X = sm.add_constant(X)
    model = sm.OLS(y, X)
    model = model.fit()
    for k in model.params:
        print(k, end='\t')
    print(model.summary())


In [3]:
def describe(df, col_name):
    target = df[col_name]
    tl = [target.mean(), target.std(), target.max()-target.min(), np.percentile(target, 25), target.median(), np.percentile(target, 75)]
    ttl = [round(x, 3) for x in tl]
    for k in ttl:
        print(k, end='\t')
    print()

## Evaluate函数是检测均值强度的函数，输入值为指定的列名

In [4]:
def evaluate(df, column='vol', add=False):
    df = df.dropna()
    MON = df['weekday'].apply(lambda x: 1 if x==0 else 0).to_numpy()
    TUE = df['weekday'].apply(lambda x: 1 if x==1 else 0).to_numpy()
    WED = df['weekday'].apply(lambda x: 1 if x==2 else 0).to_numpy()
    THU = df['weekday'].apply(lambda x: 1 if x==3 else 0).to_numpy()
    FRI = df['weekday'].apply(lambda x: 1 if x==4 else 0).to_numpy()
    y = df[column].values
    X = np.column_stack([MON, TUE, WED, THU, FRI])
    if add:
        X = sm.add_constant(X)
    model = sm.OLS(y, X)
    model = model.fit()
    for j, k in enumerate(model.params):
        print(round(k, 4))
        print("({})".format(round(model.tvalues[j], 2)))
#     print(round(model.f_pvalue, 4))
    print(round(model.rsquared_adj, 4))
    print(model.summary())

def new_evaluate(df, column='vol'):
    df = df.dropna()
    
    MON = df['weekday'].apply(lambda x: 1 if x==0 else 0).to_numpy()
    TUE = df['weekday'].apply(lambda x: 1 if x==1 else 0).to_numpy()
    WED = df['weekday'].apply(lambda x: 1 if x==2 else 0).to_numpy()
    THU = df['weekday'].apply(lambda x: 1 if x==3 else 0).to_numpy()
    FRI = df['weekday'].apply(lambda x: 1 if x==4 else 0).to_numpy()
    y = df[column].values
    X = np.column_stack([MON,TUE, WED, THU, FRI])
    model = sm.OLS(y, X)
    model = model.fit()
    for j, k in enumerate(model.params):
        print(round(k, 4))
        print("({})".format(round(model.tvalues[j], 2)))
#     print(round(model.f_pvalue, 4))
    print(round(model.rsquared_adj, 4))
    print(model.summary())
    print(model.summary())

# 开始读取数据 

In [5]:
df = pd.DataFrame()
for fn in os.listdir('td_vol/call'):
    path = 'td_vol/call/'+fn
    tmp = pd.read_csv(path)
    tmp['type'] = 'C'
    df = pd.concat([df, tmp])
    
for fn in os.listdir('td_vol/put'):
    path = 'td_vol/put/'+fn
    
    tmp = pd.read_csv(path)
    tmp['type'] = 'P'
    df = pd.concat([df, tmp])
df = df.reset_index(drop=True).dropna()
print(df[(df['to_maturity']>=6)].groupby("week_day").mean())
print(df[(df['to_maturity']>=6)].groupby("week_day").std())
print(df[(df['to_maturity']>=6)&(df['date']>20161101)].groupby("week_day").mean())
print(df[(df['to_maturity']>=6)&(df['date']>20161101)].groupby("week_day").std())

                  date   imp_vol  to_maturity  imp_change
week_day                                                 
0         2.018142e+07  0.226957    66.271210   -0.001715
1         2.018115e+07  0.226116    65.244816   -0.007227
2         2.018109e+07  0.225482    64.629259    0.000421
3         2.018118e+07  0.226192    67.052556    0.002559
4         2.018111e+07  0.226086    66.827682    0.003342
                  date   imp_vol  to_maturity  imp_change
week_day                                                 
0         19158.160201  0.091785    47.634775    0.082138
1         19348.767946  0.090573    47.432884    0.053378
2         19300.171977  0.089790    47.394727    0.045591
3         19170.930705  0.087203    46.641044    0.051900
4         19118.780598  0.088680    47.479884    0.060019
                  date   imp_vol  to_maturity  imp_change
week_day                                                 
0         2.019019e+07  0.199107    68.178538   -0.002447
1         2.01

In [6]:
describe(df[(df['to_maturity']>=6)], 'imp_change')

-0.001	0.06	1.113	-0.027	-0.003	0.021	


In [7]:
etfs = pd.read_csv('50etf_5m.csv')
etfs['time'] = etfs.date.apply(lambda x: x.split()[-1])
etfs['date'] = etfs.date.apply(lambda x: x.split()[0])
etfs = etfs[(etfs['date'] >= '2015-02-09')&(etfs['date'] <= '2021-04-14')]
etfs = etfs.reset_index(drop=True)

In [8]:
abc = pd.read_csv('50etf_5m.csv')
abc.date.apply(lambda x: x.split()[-1])
abc['time'] = abc.date.apply(lambda x: x.split()[-1])
abc['date'] = abc.date.apply(lambda x: x.split()[0])
ends = abc[abc['time'] == '15:00:00'].reset_index(drop=True)[['date', 'sectional_close']]
begins = abc[abc['time'] == '09:35:00'].reset_index(drop=True)[['date', 'sectional_open']]
ends['tomorrow_open'] = begins.sectional_open.shift(-1)
# pd.to_datetime(etfs['date']).apply(lambda x: x.weekday())

In [9]:
opens = etfs.groupby('date').apply(lambda x: x['open'].iloc[0]).reset_index()
opens['yclose'] = etfs.groupby('date').apply(lambda x: x['yclose'].iloc[0]).values
opens.columns = ['date', 'sectional_close', 'yclose']
opens['time'] = "09:30:00"

selected = etfs[['date', 'sectional_close', 'yclose', 'time']][:]
selected.loc[selected['time'] == '09:35:00', 'yclose'] = opens['sectional_close'].values
selected = pd.concat([selected, opens]).sort_values(by=['date', 'time']).reset_index(drop=True)
selected.loc[:, 'cont_change'] = np.log(selected['sectional_close'].values / selected['yclose'].values)

rvs = selected.groupby('date').apply(lambda x: np.sqrt((x['cont_change']**2).sum())*np.sqrt(252)).reset_index()
rvs.columns = ['date', 'rv']
overn = selected[selected['time']=="09:30:00"].reset_index(drop=True)
overn['on_rv'] = np.abs(overn['cont_change'])*np.sqrt(252)
overn= overn[['date', 'on_rv']]
cm = selected[selected['time']>"09:30:00"].groupby('date').apply(lambda x: np.sqrt((x['cont_change']**2).sum())*np.sqrt(252)).reset_index()
cm.columns = ['date', 'id_rv']

hvs = selected[selected['time']=="15:00:00"][:]
hvs['ret'] = hvs.sectional_close.pct_change()
hvs = hvs[['date', 'ret']][:]

hvs = (hvs.ret.rolling(30).std()*np.sqrt(252)).reset_index()
hvs['date'] = selected.date.unique()
hvs.columns = ['idx', 'HV', 'date']

rvs = pd.merge(rvs, overn, on='date')
rvs = pd.merge(rvs, cm, on='date')
# rvs = pd.merge(rvs, hvs[['HV', 'date']], on='date')

rvs['rvc'] = rvs['rv'] / rvs['rv'].shift(1) - 1
rvs['idc'] = rvs['id_rv'] - rvs['id_rv'].shift(1)
rvs['onc'] = rvs['on_rv']  - rvs['on_rv'].shift(1)

rvs['prev_rv'] = rvs['rv'].shift(1)
rvs['nxt_rv'] = rvs['rv'].shift(-1)
rvs['prev_id'] = rvs['id_rv'].shift(1)
rvs['prev_on'] = rvs['on_rv'].shift(1)
rvs['prev_rvc'] = rvs['rvc'].shift(1)
rvs['prev_idc'] = rvs['idc'].shift(1)
rvs['prev_onc'] = rvs['onc'].shift(1)

In [10]:
calls = df[df['type']=='C'].dropna()
datetmp = pd.to_datetime(calls.date.apply(str)).astype(str)
calls = calls[(calls['to_maturity']>=6)]
# calls = calls.sort_values(by=['date', 'to_maturity']).drop_duplicates(subset=['date'])
calls['date'] = datetmp
calls['ivc'] = calls.imp_vol.values * calls.imp_change.values / (1 + calls.imp_change.values)
calls = calls.drop(columns=['type']).sort_values(by='date').dropna().reset_index(drop=True)
calls.columns = ['date', 'iv', 'to_maturity', 'ic_pct', 'weekday', 'imp_change']
all_vols = pd.merge(calls, rvs, on='date')
all_vols['vrp'] =  all_vols['rv']**2 - all_vols['iv']**2
all_vols['year'] = all_vols.date.apply(lambda x: x.split('-')[0] if '2021' not in x else '2020')
all_vols['date'] = all_vols['date'].apply(lambda x: int(x.replace("-", "")))
all_vols = all_vols[all_vols['date']>=20161101]
all_vols = all_vols.dropna().reset_index(drop=True)
all_vols.head()

,date,iv,to_maturity,ic_pct,weekday,imp_change,rv,on_rv,id_rv,rvc,...,onc,prev_rv,nxt_rv,prev_id,prev_on,prev_rvc,prev_idc,prev_onc,vrp,year
0,20190426,0.247822,164.0,-0.026581,4,-0.006767,0.194126,0.071003,0.180676,0.231736,...,0.001083,0.157604,0.232370,0.141245,0.069920,-0.287231,-0.075688,0.027117,-0.023731,2019
1,20190429,0.245190,163.0,-0.010621,0,-0.002632,0.232370,0.044035,0.228160,0.197005,...,-0.026968,0.194126,0.169488,0.180676,0.071003,0.231736,0.039430,0.001083,-0.006122,2019
2,20190430,0.242573,162.0,-0.010673,1,-0.002617,0.169488,0.005436,0.169401,-0.270610,...,-0.038599,0.232370,0.583703,0.228160,0.044035,0.197005,0.047484,-0.026968,-0.030115,2019
3,20190506,0.254429,161.0,0.048876,0,0.011856,0.583703,0.484419,0.325649,2.443910,...,0.478984,0.169488,0.226320,0.169401,0.005436,-0.270610,-0.058758,-0.038599,0.275975,2019
4,20190507,0.241001,160.0,-0.052776,1,-0.013428,0.226320,0.073875,0.213924,-0.612268,...,-0.410545,0.583703,0.310666,0.325649,0.484419,2.443910,0.156248,0.478984,-0.006861,2019


## Simple Regressions

In [11]:
vol_df = all_vols[:]
IV = vol_df.iv.values
RV = vol_df.rv.values
RV1 = vol_df.prev_rv.values
DTM = np.log(vol_df.to_maturity.values / 252)
X = np.column_stack([RV, RV1, DTM])
model = sm.OLS(IV, sm.add_constant(X))
model = model.fit()
for j, k in enumerate(model.params):
    print(round(k, 4))
    print("({})".format(round(model.tvalues[j], 2)))
#     print(round(model.f_pvalue, 4))
print(round(model.rsquared_adj, 4))
print(model.summary())

0.1642
(60.05)
0.189
(17.87)
0.1474
(13.76)
0.0086
(7.97)
0.3836
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.385
Model:                            OLS   Adj. R-squared:                  0.384
Method:                 Least Squares   F-statistic:                     312.0
Date:                Tue, 07 Mar 2023   Prob (F-statistic):          2.74e-157
Time:                        22:59:54   Log-Likelihood:                 2852.9
No. Observations:                1500   AIC:                            -5698.
Df Residuals:                    1496   BIC:                            -5677.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [12]:
vol_df = all_vols[:]
IV = vol_df.imp_change.values
RV = vol_df.rvc.values
RV1 = vol_df.prev_rvc.values
DTM = 252 / vol_df.to_maturity.values
X = np.column_stack([RV, RV1, DTM])
model = sm.OLS(IV, sm.add_constant(X))
model = model.fit()
for j, k in enumerate(model.params):
    print(round(k, 4))
    print("({})".format(round(model.tvalues[j], 2)))
#     print(round(model.f_pvalue, 4))
print(round(model.rsquared_adj, 4))
print(model.summary())

-0.0005
(-1.07)
0.0089
(12.35)
-0.0002
(-0.31)
-0.0001
(-1.93)
0.1061
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.108
Model:                            OLS   Adj. R-squared:                  0.106
Method:                 Least Squares   F-statistic:                     60.30
Date:                Tue, 07 Mar 2023   Prob (F-statistic):           8.46e-37
Time:                        22:59:55   Log-Likelihood:                 4438.7
No. Observations:                1500   AIC:                            -8869.
Df Residuals:                    1496   BIC:                            -8848.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------

### 细分拆解：日内与隔夜

In [13]:
vol_df = all_vols[:]
IV = vol_df.imp_change.values
RV = vol_df.rv.values
RV1 = vol_df.prev_rv.values
DTM = np.sqrt(vol_df.to_maturity.values / 252)
X = np.column_stack([ RV, RV1, DTM])
model = sm.OLS(IV, sm.add_constant(X))
model = model.fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.178
Model:                            OLS   Adj. R-squared:                  0.177
Method:                 Least Squares   F-statistic:                     108.3
Date:                Tue, 07 Mar 2023   Prob (F-statistic):           1.82e-63
Time:                        22:59:56   Log-Likelihood:                 4500.5
No. Observations:                1500   AIC:                            -8993.
Df Residuals:                    1496   BIC:                            -8972.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0042      0.001     -3.910      0.0

In [14]:
IV = all_vols.imp_change.values[1:]
# RV = all_vols.rvc.values[1:]
RV2 = all_vols.idc.values[:-1]
# HV = all_vols.hvc.values
X = np.column_stack([RV2])
model = sm.OLS(IV, sm.add_constant(X))
model = model.fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.027
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     41.70
Date:                Tue, 07 Mar 2023   Prob (F-statistic):           1.44e-10
Time:                        22:59:56   Log-Likelihood:                 4370.4
No. Observations:                1499   AIC:                            -8737.
Df Residuals:                    1497   BIC:                            -8726.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0004      0.000     -1.076      0.2

## VRP + Delta-neutral Return

In [15]:
def orz(df, col, ycol):
    df = df.dropna()
    MON = df['weekday'].apply(lambda x: 1 if x==4 else 0)*df[col]
    TUE = df['weekday'].apply(lambda x: 1 if x==0 else 0)*df[col]
    WED = df['weekday'].apply(lambda x: 1 if x==1 else 0)*df[col]
    THU = df['weekday'].apply(lambda x: 1 if x==2 else 0)*df[col]
    FRI = df['weekday'].apply(lambda x: 1 if x==3 else 0)*df[col]
    y = df[ycol].values
    X = np.column_stack([MON,TUE, WED, THU, FRI])
    model = sm.OLS(y, sm.add_constant(X))
    model = model.fit()
    for j, k in enumerate(model.params):
        print(round(k, 4))
        print("({})".format(round(model.tvalues[j], 2)))
#     print(round(model.f_pvalue, 4))
    print(round(model.rsquared_adj, 4))
    print(model.summary())


In [16]:
calls = df[df['type']=='C'].dropna()
datetmp = pd.to_datetime(calls.date.apply(str)).astype(str)
calls = calls[(calls['to_maturity']>=6)]
# calls = calls.sort_values(by=['date', 'to_maturity']).drop_duplicates(subset=['date'])
calls['date'] = datetmp
calls['ivc'] = calls.imp_vol.values * calls.imp_change.values / (1 + calls.imp_change.values)
calls = calls.drop(columns=['type']).sort_values(by='date').dropna().reset_index(drop=True)
calls.columns = ['date', 'iv', 'to_maturity', 'ic_pct', 'weekday', 'imp_change']
all_vols = pd.merge(calls, rvs, on='date')
all_vols['vrp'] =  all_vols['rv']**2 - all_vols['iv']**2
all_vols['year'] = all_vols.date.apply(lambda x: x.split('-')[0] if '2021' not in x else '2020')
all_vols['date'] = all_vols['date'].apply(lambda x: int(x.replace("-", "")))
all_vols = all_vols[all_vols['date']>=20161101]
all_vols = all_vols.dropna().reset_index(drop=True)

all_vols['iv2'] = all_vols.iv**2
all_vols['rv2'] = all_vols.rv**2
all_vols['nrv2'] = all_vols.nxt_rv**2
all_vols.mean()

all_vols['vrp'] = -all_vols['iv2'] + all_vols['rv2']

a = pd.read_csv("all_long_call_new.csv")
a['date'] = a['date'].apply(lambda x: int("".join(x.split("-"))))
tmp = a[['date', 'neutral_pnl', 'dtm']]
tmp.columns = ['date', 'neutral_ret', 'to_maturity']
lc = pd.merge(all_vols[['date', 'vrp', "to_maturity", 'weekday']], tmp ,on=['date', 'to_maturity'])
model = sm.OLS(lc.neutral_ret, sm.add_constant(lc.vrp))
model = model.fit()
print(model.summary())
a = pd.read_csv("all_long_put_new.csv")
a['date'] = a['date'].apply(lambda x: int("".join(x.split("-"))))
tmp = a[['date', 'neutral_pnl', 'dtm']]
tmp.columns = ['date', 'neutral_ret', 'to_maturity']
lp = pd.merge(all_vols[['date', 'vrp', "to_maturity", 'weekday']], tmp ,on=['date', 'to_maturity'])
model = sm.OLS(lp.neutral_ret, sm.add_constant(lp.vrp))
model = model.fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            neutral_ret   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     2.194
Date:                Tue, 07 Mar 2023   Prob (F-statistic):              0.139
Time:                        22:59:58   Log-Likelihood:                 1357.9
No. Observations:                1475   AIC:                            -2712.
Df Residuals:                    1473   BIC:                            -2701.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.572e-05      0.003      0.018      0.9

In [17]:
calls = df[df['type']=='C'].dropna()
datetmp = pd.to_datetime(calls.date.apply(str)).astype(str)
calls = calls[(calls['to_maturity']>=6)]
# calls = calls.sort_values(by=['date', 'to_maturity']).drop_duplicates(subset=['date'])
calls['date'] = datetmp
calls['ivc'] = calls.imp_vol.values * calls.imp_change.values / (1 + calls.imp_change.values)
calls = calls.drop(columns=['type']).sort_values(by='date').dropna().reset_index(drop=True)
calls.columns = ['date', 'iv', 'to_maturity', 'ic_pct', 'weekday', 'imp_change']
all_vols = pd.merge(calls, rvs, on='date')
all_vols['vrp'] =  all_vols['rv']**2 - all_vols['iv']**2
all_vols['year'] = all_vols.date.apply(lambda x: x.split('-')[0] if '2021' not in x else '2020')
all_vols['date'] = all_vols['date'].apply(lambda x: int(x.replace("-", "")))
all_vols = all_vols[all_vols['date']>=20161101]
all_vols = all_vols.dropna().reset_index(drop=True)

all_vols['iv2'] = all_vols.iv**2
all_vols['rv2'] = all_vols.rv**2
all_vols['nrv2'] = all_vols.nxt_rv**2
all_vols.mean()

all_vols['vrp'] = all_vols['rv2'] - all_vols['iv2']

a = pd.read_csv("all_long_call_new.csv")
a['date'] = a['date'].apply(lambda x: int("".join(x.split("-"))))
dates = np.array([20150208] + a.date.unique().tolist())
rets = a[['date', 'dtm', 'neutral_pnl']][:]
rets.columns = ['last_day', 'dtm', 'last_ret']
rets.loc[:, "dtm"] = rets.dtm.apply(lambda x: x - 1).values
a['last_day'] = a.date.apply(lambda x: dates[np.where(dates==x)[0][0]-1])
a = a.merge(rets)
tmp = a[['date', 'neutral_pnl', 'dtm', 'last_ret']]
tmp.columns = ['date', 'neutral_ret', 'to_maturity', 'last_ret']
lp = pd.merge(all_vols[['date', 'vrp', "to_maturity", 'weekday']], tmp ,on=['date', 'to_maturity'])
X = np.column_stack([lp.vrp, lp.last_ret])
model = sm.OLS(lp.neutral_ret, sm.add_constant(X))
model = model.fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            neutral_ret   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.336
Date:                Tue, 07 Mar 2023   Prob (F-statistic):              0.263
Time:                        22:59:59   Log-Likelihood:                 1358.1
No. Observations:                1475   AIC:                            -2710.
Df Residuals:                    1472   BIC:                            -2694.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -5.117e-05      0.003     -0.020      0.9

In [18]:
a = pd.read_csv("all_long_put_new.csv")
a['date'] = a['date'].apply(lambda x: int("".join(x.split("-"))))
rets = a[['date', 'dtm', 'neutral_pnl']][:]
rets.columns = ['last_day', 'dtm', 'last_ret']
rets.loc[:, "dtm"] = rets.dtm.apply(lambda x: x - 1).values
a['last_day'] = a.date.apply(lambda x: dates[np.where(dates==x)[0][0]-1])
a = a.merge(rets)
tmp = a[['date', 'neutral_pnl', 'dtm', 'last_ret']]
tmp.columns = ['date', 'neutral_ret', 'to_maturity', 'last_ret']
lc = pd.merge(all_vols[['date', 'vrp', "to_maturity", 'weekday']], tmp ,on=['date', 'to_maturity'])
X = np.column_stack([lc.vrp, lc.last_ret])
model = sm.OLS(lc.neutral_ret, sm.add_constant(X))
model = model.fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            neutral_ret   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     2.031
Date:                Tue, 07 Mar 2023   Prob (F-statistic):              0.132
Time:                        23:00:00   Log-Likelihood:                 1424.8
No. Observations:                1475   AIC:                            -2844.
Df Residuals:                    1472   BIC:                            -2828.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0022      0.002     -0.911      0.3

In [19]:
for k in all_vols.groupby('weekday').std().iv.values:
    print(round(k,4), end='\n')

0.0487
0.0459
0.045
0.0444
0.0462


In [20]:
model = sm.OLS(lc.neutral_ret, sm.add_constant(lc.vrp))
model = model.fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            neutral_ret   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.9684
Date:                Tue, 07 Mar 2023   Prob (F-statistic):              0.325
Time:                        23:00:04   Log-Likelihood:                 1423.3
No. Observations:                1475   AIC:                            -2843.
Df Residuals:                    1473   BIC:                            -2832.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0021      0.002     -0.860      0.3

## 处理straddle:對齊

In [21]:
straddles = pd.read_csv('all_straddles_td.csv')
straddles = straddles[straddles['date']>='2016-11-01']
straddles = straddles[straddles['dtm']>=6]
# straddles = straddles.drop_duplicates(subset='date').reset_index(drop=True)
straddles['principal'] = straddles['call_close'] / straddles['call_delta'] - straddles['put_close'] / straddles['put_delta']

close_to_close = straddles[['date', 'exe_date', 'weekday', 'dtm', 'call_delta', 'call_close', 'call_next_close', 'put_delta', 'put_close', 'put_next_close', 'principal']]
close_to_open = straddles[['date', 'exe_date','weekday', 'dtm', 'call_delta', 'call_close', 'call_next_open', 'put_delta', 'put_close', 'put_next_open', 'principal']]
open_to_close = straddles[['date', 'exe_date','weekday', 'dtm', 'call_delta', 'call_next_close', 'call_next_open', 'put_delta', 'put_next_close', 'put_next_open', 'principal']]

close_to_close['call_ret'] = close_to_close['call_next_close'].values - close_to_close['call_close'].values

close_to_close['put_ret'] = close_to_close['put_next_close'].values - close_to_close['put_close'].values

close_to_close['ret'] = close_to_close['call_ret'] / close_to_close['call_delta'] - close_to_close['put_ret'] / close_to_close['put_delta'] 


close_to_close['neutral_pnl'] = close_to_close['ret'] / close_to_close['principal']


tmp = close_to_close[['date', 'neutral_pnl', 'dtm']][:]
tmp['date'] = tmp.date.apply(lambda x: int(x.replace("-","")))
tmp.columns = ['date', 'neutral_ret', 'to_maturity']
straddles = pd.merge(all_vols[['date', 'vrp', "to_maturity", 'weekday']], tmp, on=['date', 'to_maturity'])
straddles

model = sm.OLS(straddles.neutral_ret, sm.add_constant(straddles.vrp))
model = model.fit()
print(model.summary())
orz(straddles, "vrp", "neutral_ret")

                            OLS Regression Results                            
Dep. Variable:            neutral_ret   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.465
Date:                Tue, 07 Mar 2023   Prob (F-statistic):              0.226
Time:                        23:00:05   Log-Likelihood:                 1451.8
No. Observations:                1496   AIC:                            -2900.
Df Residuals:                    1494   BIC:                            -2889.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0008      0.002     -0.323      0.7

C:\Users\21130\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [31]:
close_to_close

,date,exe_date,weekday,dtm,call_delta,call_close,call_next_close,put_delta,put_close,put_next_close,principal,call_ret,put_ret,ret,neutral_pnl
1656,2016-11-01,2016-11-23,1,17.0,0.588533,0.03635,0.02815,-0.411467,0.02155,0.02980,0.114137,-0.00820,0.00825,0.006117,0.053596
1657,2016-11-01,2016-12-28,1,42.0,0.544992,0.05345,0.04600,-0.455008,0.04300,0.04975,0.192579,-0.00745,0.00675,0.001165,0.006049
1658,2016-11-01,2017-03-22,1,96.0,0.518810,0.08265,0.07490,-0.481190,0.08230,0.08995,0.330341,-0.00775,0.00765,0.000960,0.002906
1659,2016-11-01,2017-06-28,1,161.0,0.524849,0.11440,0.10980,-0.475151,0.11480,0.12215,0.459575,-0.00460,0.00735,0.006704,0.014588
1660,2016-11-02,2016-11-23,2,16.0,0.497247,0.02815,0.04285,-0.502753,0.02980,0.02235,0.115885,0.01470,-0.00745,0.014744,0.127232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5736,2021-04-12,2021-09-22,0,112.0,0.552583,0.19860,0.19030,-0.447417,0.16920,0.17550,0.737574,-0.00830,0.00630,-0.000940,-0.001274
5737,2021-04-13,2021-04-28,1,12.0,0.383414,0.03180,0.03000,-0.616586,0.07000,0.06600,0.196467,-0.00180,-0.00400,-0.011182,-0.056915
5738,2021-04-13,2021-05-26,1,29.0,0.590323,0.10910,0.11440,-0.409677,0.06660,0.06240,0.347381,0.00530,-0.00420,-0.001274,-0.003667
5739,2021-04-13,2021-06-23,1,48.0,0.487063,0.10570,0.11180,-0.512937,0.12760,0.11900,0.465779,0.00610,-0.00860,-0.004242,-0.009108


In [22]:
a = close_to_close
rets = a[['date', 'dtm', 'neutral_pnl']][:]
rets.columns = ['last_day', 'dtm', 'last_ret']
rets.loc[:, "dtm"] = rets.dtm.apply(lambda x: x - 1).values
dates = np.array([20211031]+a.date.unique().tolist())
a['last_day'] = a.date.apply(lambda x: dates[np.where(dates==x)[0][0]-1])
a = a.merge(rets)
tmp = a[['date', 'neutral_pnl', 'dtm', 'last_ret']]
tmp.columns = ['date', 'neutral_ret', 'to_maturity', 'last_ret']
tmp.date=tmp.date.apply(lambda x: int("".join(x.split("-"))))
lc = pd.merge(all_vols[['date', 'vrp', "to_maturity", 'weekday']], tmp ,on=['date', 'to_maturity'])
X = np.column_stack([lc.vrp, lc.last_ret])
model = sm.OLS(lc.neutral_ret, sm.add_constant(X))
model = model.fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            neutral_ret   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.461
Date:                Tue, 07 Mar 2023   Prob (F-statistic):              0.232
Time:                        23:00:06   Log-Likelihood:                 1452.6
No. Observations:                1496   AIC:                            -2899.
Df Residuals:                    1493   BIC:                            -2883.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0009      0.002     -0.375      0.7

C:\Users\21130\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
